# Test Generators
If you want to create a list of tests, you can do it with a list comprehension.
As an example we create tests for quantiles.

But before that we prepare reference and current test data:

In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml


data = fetch_openml(name='adult', version=2, as_frame='auto')
data.frame['target'] = data.frame['education-num']
data.frame['preds'] = data.frame['education-num'].values + np.random.normal(0, 6, data.frame.shape[0])
reference_data = data.frame[:20000]
current_data = data.frame[20000:]

Now we can import the quantiles test and test suite classes from **evidently** and add a list of test with different quantiles to our suite:

In [ ]:
from evidently.tests import TestValueQuantile
from evidently.test_suite import TestSuite


suite = TestSuite(tests=[
    TestValueQuantile(column_name="education-num", quantile=quantile) for quantile in [0.5, 0.9, 0.99]
])

suite.run(current_data=current_data, reference_data=reference_data)
suite

Or you can do it for a list with specific columns

In [ ]:
from evidently.tests import TestFeatureValueMin


# test that values in specified columns are positive (greater than zero condition)
suite = TestSuite(tests=[
    TestFeatureValueMin(column_name=column_name, gt=0) for column_name in ["age", "fnlwgt", "education-num"]
])

suite.run(current_data=current_data, reference_data=reference_data)
suite

But if you want to use column names in tests generation, sometimes you can get them before the suite calculation is already launched.

In this case you can use test generators:

In [ ]:
from evidently.tests.base_test import generate_columns_tests

from evidently.tests import TestColumnNANShare

suite = TestSuite(tests=[generate_columns_tests(TestColumnNANShare)])
suite.run(current_data=current_data, reference_data=reference_data)
suite.show()

By default, it generates tests with no parameters for all columns.
If you want to set custom parameters for all generated tests, you can do it with a separate parameter

In [ ]:
# set condition for all generated tests and specify that we want to generate tests for all columns explicitly
suite = TestSuite(tests=[generate_columns_tests(TestColumnNANShare, columns="all", parameters={"lt": 0.5})])
suite.run(current_data=current_data, reference_data=reference_data)
suite.show()

If you want to generate tests for numeric features only, you can set parameter `columns` to "num"

In [ ]:
suite = TestSuite(tests=[generate_columns_tests(TestFeatureValueMin, columns="num")])
suite.run(current_data=current_data, reference_data=reference_data)
suite.show()

If you want to generate tests for category features only, you can set parameter `columns` to "cat"

In [ ]:
suite = TestSuite(tests=[generate_columns_tests(TestColumnNANShare, columns="cat", parameters={"lt": 0.1})])
suite.run(current_data=current_data, reference_data=reference_data)
suite.show()